# Preparation

The following sections assume that these values are properly set in `.env` or your env vars.

In [ ]:
import { load } from "https://deno.land/std@0.202.0/dotenv/mod.ts";
await load({ export: true });

const organizationId = Deno.env.get("DEPLOY_ORGANIZATION_ID")!;
const projectId = Deno.env.get("DEPLOY_PROJECT_ID")!;
const token = Deno.env.get("DEPLOY_ORGANIZATION_ACCESS_TOKEN")!;

# Create a deployment

A typical flow for creating a deployment is described in this section.

## 1. Initiate a build

First of all, you will call `POST /v1/projects/:projectId/deployments` to initiate a build, which gives
you the deployment ID.

In [ ]:
import { assertEquals } from "https://deno.land/std@0.202.0/testing/asserts.ts";

const resp = await fetch(
  `https://api.deno.com/v1/projects/${projectId}/deployments`,
  {
    method: "POST",
    headers: {
      authorization: `Bearer ${token}`,
      "content-type": "application/json",
    },
    body: JSON.stringify(
      {
        "entryPointUrl": "main.ts",
        "importMapUrl": null,
        "lockFileUrl": null,
        "compilerOptions": null,
        "assets": {
          "main.ts": {
            "kind": "file",
            "content":
              "Deno.serve((req: Request) => new Response('Hello World'));",
          },
        },
        "envVars": {
          "FOO": "foo",
        },
      } satisfies CreateDeploymentRequest,
    ),
  },
);
assertEquals(resp.status, 200);

const deployment: DeploymentResponse = await resp.json();
console.log(deployment);

## 2. Wait for the build to be completed

Build of a deployment is an async process; in order to make sure it is completed,
you have to check the current status, by either:

- watching the stream of the progress, or
- calling the deployment detail API (maybe periodically)

In [ ]:
//////////////////////////////////////////////////
//////////////////////////////////////////////////
// watch the stream of build logs
//////////////////////////////////////////////////
//////////////////////////////////////////////////

import { assertEquals } from "https://deno.land/std@0.202.0/testing/asserts.ts";
import { DelimiterStream } from "https://deno.land/std@0.202.0/streams/delimiter_stream.ts";

const resp = await fetch(
  `https://api.deno.com/v1/deployments/${deployment.id}/build_logs`,
  {
    method: "GET",
    headers: {
      authorization: `Bearer ${token}`,
      // application/json or application/x-ndjson is supported. The latter is 
      // more friendly for streaming
      accept: "application/x-ndjson",
    },
  },
);
assertEquals(resp.status, 200);

const lines = resp.body!.pipeThrough(new DelimiterStream(new TextEncoder().encode("\n")))
.pipeThrough(new TextDecoderStream());

console.log("-------- Build logs start --------");
for await (const line of lines) {
  const buildLogLine: BuildLogLine = JSON.parse(line);
  console.log(buildLogLine);
}

// Reaching to this point means the build has finsihed 
console.log("-------- Build logs end --------");

In [ ]:
//////////////////////////////////////////////////
//////////////////////////////////////////////////
// get the deployment detail info
//////////////////////////////////////////////////
//////////////////////////////////////////////////

import { assertEquals } from "https://deno.land/std@0.202.0/testing/asserts.ts";

const resp = await fetch(
  `https://api.deno.com/v1/deployments/${deployment.id}`,
  {
    method: "GET",
    headers: {
      authorization: `Bearer ${token}`,
    },
  },
);
assertEquals(resp.status, 200);

const deployment: DeploymentResponse = await resp.json();
console.log(deployment);

// status === "success" means your deployment is ready
if (deployment.status === "success" && deployment.domains[0] !== undefined) {
  const deploymentResp = await fetch(`https://${deployment.domains[0]}`);
  assertEquals(deploymentResp.status, 200);
  assertEquals(await deploymentResp.text(), "Hello World");
}

# Register and verify a domain, then provision TLS certificates

The next demo outlines how to perform domain-related operations, where we register a domain and verify its ownership, then provision TLS certificates for the domain.

## 1. Register a domain

Firstly, call `POST /v1/organizations/${organizationId}/domains` to register a domain. Refer to the below sample snippet, though make sure to substitute the domain value with yours.

In [ ]:
import { assertEquals } from "https://deno.land/std@0.202.0/testing/asserts.ts";

const resp = await fetch(
  `https://api.deno.com/v1/organizations/${organizationId}/domains`,
  {
    method: "POST",
    headers: {
      authorization: `Bearer ${token}`,
      "content-type": "application/json",
    },
    body: JSON.stringify(
      {
        // ⚠️REPLACE THIS WITH YOURS!
        "domain": "deploy-sample1.maguro.me",
      } satisfies CreateDomainRequest,
    ),
  },
);
assertEquals(resp.status, 200);

const domain: DomainResponse = await resp.json();
console.log(domain);

## 2. Verify the domain ownership

Once you registered your domain, the domain needs to be verified that you are the owner of the domain. You can do so by following these steps:

1. Retrieve the `dnsRecords` field from the response body of the API call above. Notice that it includes three values: type `A`, `AAAA`, and `CNAME`.
2. Go to the dashboard (or something like this) where you can manage DNS records for your domain, and add DNS three records you got. Here's my setting on Cloudflare dashboard:

![How I configured DNS records on Cloudflare dashboard](https://github.com/magurotuna/zig-deque/assets/23649474/5bfeaafc-58c0-46fa-857c-ec33fdf3f3a8)


3. Then call the API to get Deploy to verify the expected records are set:

In [ ]:

import { assertEquals } from "https://deno.land/std@0.202.0/testing/asserts.ts";

const resp = await fetch(
  `https://api.deno.com/v1/domains/${domain.id}/verify`,
  {
    method: "POST",
    headers: {
      authorization: `Bearer ${token}`,
    },
  },
);
assertEquals(resp.status, 200);


## 3. Provision TLS certificates

Now it's verified that you are the owner of the domain. Final step is to provision TLS certificates for the domain. You can either upload the certificates you prepared somehow, or just leave the certificates preparation to Deploy. Here we will demonstrate the latter, auto-provisioning capability.

All you have to do is to call `POST /v1/domains/:domainId/certificates/provision` which triggers the auto-provisioning step for the specified domain.

In [ ]:
import { assertEquals } from "https://deno.land/std@0.202.0/testing/asserts.ts";

// ⚠️ This call may take long, like 20 seconds or so.
const resp = await fetch(
  `https://api.deno.com/v1/domains/${domain.id}/certificates/provision`,
  {
    method: "POST",
    headers: {
      authorization: `Bearer ${token}`,
    },
  },
);
assertEquals(resp.status, 200);

Just to make sure, let's call `GET /v1/domains/:domainId` to see the current state of the domain.

In [ ]:
import { assert, assertEquals } from "https://deno.land/std@0.202.0/testing/asserts.ts";

const resp = await fetch(
  `https://api.deno.com/v1/domains/${domain.id}`,
  {
    method: "GET",
    headers: {
      authorization: `Bearer ${token}`,
    },
  },
);
assertEquals(resp.status, 200);

const domain: DomainResponse = await resp.json();
// Provisioning process succeeded
assertEquals(domain.provisioningStatus.code, "success");
const certCipherSet = new Set(domain.certificates.map((cert) => cert.cipher));
// Both ECDSA and RSA certificates have been provisioned
assertEquals(certCipherSet.size, 2);
assert(certCipherSet.has("ec"));
assert(certCipherSet.has("rsa"));

# Associate the custom domain with the deployment

Although the domain setup has completed, it is of no use until it gets connected to an actual deployment. We accomplish it by calling `PATCH /v1/domains/:domainId`.

In [ ]:
import { assertEquals } from "https://deno.land/std@0.202.0/testing/asserts.ts";

const resp = await fetch(
  `https://api.deno.com/v1/domains/${domain.id}`,
  {
    method: "PATCH",
    headers: {
      authorization: `Bearer ${token}`,
      "content-type": "application/json",
    },
    body: JSON.stringify(
      {
        "deploymentId": deployment.id,
      } satisfies UpdateDomainAssociationRequest,
    ),
  },
);
assertEquals(resp.status, 200);

Now you can confirm that your deployment is reachable using your own domain. Here's the result of `curl` in my case:

```sh
❯ curl --include 'https://deploy-sample1.maguro.me'
HTTP/2 200
content-type: text/plain;charset=UTF-8
vary: Accept-Encoding
date: Thu, 21 Sep 2023 10:51:34 GMT
content-length: 11
via: http/2 edgeproxy
server: deno/gcp-asia-northeast1

Hello World%
```

# Appendix

Here is a list of types used in the sample code snippets above.

In [ ]:
type CreateDeploymentRequest = {
  entryPointUrl: string;
  importMapUrl: string | null;
  lockFileUrl: string | null;
  compilerOptions: CompilerOptions | null;
  assets: Record<string, Asset>;
  envVars: Record<string, string>;
};

type CompilerOptions = {
  jsx: string | null;
  jsxFactory: string | null;
  jsxFragmentFactory: string | null;
  jsxImportSource: string | null;
};

type Asset = File | Symlink;

type File = FileContent | FileHash;

type FileContent = {
  kind: "file";
  content: string;
  encoding: "utf-8" | "base64";
};

type FileHash = {
  kind: "file";
  gitSha1: string;
};

type Symlink = {
  kind: "symlink";
  target: string;
};

type DeploymentResponse = {
  id: string;
  projectId: string;
  status: "failed" | "pending" | "success";
  domains?: string[];
  createdAt: string;
  updatedAt: string;
};

type BuildLogLine = {
  level: string;
  message: string;
};

type CreateDomainRequest = {
  domain: string;
};

type DomainResponse = {
  id: string;
  organizationId: string;
  domain: string;
  token: string;
  isValidated: boolean;
  certificates: DomainCertificate[];
  provisioningStatus: ProvisioningStatus;
  projectId: string | null;
  createdAt: string;
  updatedAt: string;
  dnsRecords: DnsRecord[];
};

type ProvisioningStatus =
  | ProvisioningSuccess
  | ProvisioningFailed
  | ProvisioningPending
  | ProvisioningManual;

type ProvisioningSuccess = {
  code: "success";
};

type ProvisioningFailed = {
  code: "failed";
  message: string;
};

type ProvisioningPending = {
  code: "pending";
};

type ProvisioningManual = {
  code: "manual";
};

type DomainCertificate = {
  cipher: "ec" | "rsa";
  expiresAt: string;
  createdAt: string;
  updatedAt: string;
};

type DnsRecord = {
  type: string;
  name: string;
  content: string;
};

type UpdateDomainAssociationRequest = {
  deploymentId: string;
};